## Layers and hodoscopes


The default *layer* class `PanelDetectorLayer` in TomOpt uses multiple *panels* `DetectorPanel` to record muon position.

`PanelDetectorLayer` inherits from the `AbsDetectorLayer` and **MUST** provide the following **methods**:

 - `forward`: The forward method to propagate the muons. It is a central aspect of `nn.Module`.
 - `get_cost`
 - `conform_detector`
  - `assign_budget`

and the following **features**:

 - `pos`: the position of the layer, either `above` or `bellow`. 
 - `lw`: the length and width of the layer.
 - `z`: the z position of the top of gthe layer. 
 - `size`: the heigth of the layer such that z - size is the bottom of the layer.
 - `device`: torch.device.

In [1]:
from torch import Tensor
# from torch import nn
from hodoscope import Hodoscope
from hodoscopelayer import HodoscopeDetectorLayer

hod1 = Hodoscope(init_xyz= [.5, .5, 1.], 
                 init_xyz_span = [1., 1., .3], 
                 xyz_gap = [.1, .1, .1])

hod2 = Hodoscope(init_xyz = [0.25, 0.25, 0.6], 
                 init_xyz_span = [1., 1., .3], 
                 xyz_gap = [.1, .1, .1])


hods = [hod1, hod2]

hod_detector_above = HodoscopeDetectorLayer(pos = 'above', lw = Tensor([1,1]), z = 1., size = 0.5, 
                                            hodoscopes = hods)

